# Problem 1: Authentication Attacks and Password Cracking

### Intro

To perform effective password cracking and authentication attacks, we used the widely known wordlist rockyou.txt, which contains millions of real-world passwords leaked from the 2009 RockYou data breach. This list is frequently used in penetration testing due to its high success rate in matching weak or reused passwords.

John the Ripper, Hashcat, and Hydra are powerful password-cracking tools widely used in cybersecurity and penetration testing.
John the Ripper is a fast, command-line tool designed to detect weak passwords by using dictionary, brute-force, and hybrid attacks.
Hashcat, on the other hand, is a more advanced and flexible GPU-accelerated password recovery tool that supports a wide range of hash algorithms and attack modes.
Both tools are commonly used to audit password security and recover plaintext passwords from hashed values.
Hydra complements these tools by performing brute-force attacks on login forms and network services, supporting multiple protocols like HTTP, FTP, and SSH. It is especially useful for testing the strength of authentication mechanisms in web applications such as DVWA.

The password hashes stored in DVWA’s database are encoded using the MD5 hashing algorithm. MD5 is a cryptographic hash function that produces a fixed 128-bit hash from any input. Since MD5 is a one-way function, we cannot reverse it directly — instead, we use tools like John the Ripper and Hashcat to attempt to recover the original plaintext passwords by comparing the hashes to known passwords from a wordlist.

![Hash passwords](imag/1_1.jpeg)

### John

![Hash passwords](imag/Hashes_john.jpeg)

We accessed the DVWA MariaDB database and extracted usernames along with their corresponding password hashes from the users table. These hashes are stored using the MD5 algorithm, which produces a 32-character hexadecimal string. We saved the extracted hashes into a file (hashes.txt) to use in password-cracking tools.

Next, we used John the Ripper with the rockyou.txt wordlist to crack the MD5 hashes. By specifying the format as raw-md5, John successfully recovered four weak passwords: password, abc123, letmein, and charley. This demonstrates how easily unsalted MD5 hashes can be cracked when users choose commonly used passwords.

### Hashcat

![Hash passwords](imag/hashcat_1.jpeg)

This tells Hashcat to crack MD5 hashes (-m 0) using the rockyou.txt wordlist against the hashes in hashes.txt.

![Hash passwords](imag/hashcat_2.jpeg)

The rockyou.txt wordlist has been successfully loaded into memory. Hashcat identifies 1,434,392 potential passwords and prepares for the cracking process. Various optimizations (e.g., Zero-Byte, Not-Salted, Raw-Hash) are applied to improve performance.

![Hash passwords](imag/hashcat_3.jpeg)

Hashcat successfully cracks 4 out of 4 hashes using the rockyou.txt wordlist. The cracked passwords (password, abc123, letmein, charley) are printed next to their corresponding MD5 hashes. The session shows 100% success in recovering all hashes, confirming that the original passwords were weak and easily guessable.

![Hash passwords](imag/hashcat_4.jpeg)

This shows a detailed summary of the cracking session. It confirms that all 4 hashes were recovered with a speed of **140.6 kH/s**. The output also displays progress, performance metrics, restore points, and candidate password suggestions. The process completed successfully in just a few seconds, clearly demonstrating **Hashcat’s efficiency** in cracking weak MD5 hashes using a dictionary attack.

### Custom wordlist and Rule Sets created

![Hash passwords](imag/Customw.jpeg)

We created a **custom password wordlist** named `custom_passwords.txt` containing common and predictable passwords that might be used in the DVWA environment. Examples include simple passwords like `123456` and `password123`, as well as more contextual ones such as `dvwa`, `admin`, `admin123`, `user123`, and `Hello123`. This wordlist is tailored to improve the efficiency of dictionary attacks by focusing on likely choices based on the application's context and typical user behavior.


![Hash passwords](imag/rulesets.jpeg)

We defined a **custom password rule set** in a file named `custom_passwordrules.txt`. These rules apply transformations to each word in the wordlist to generate realistic password variations. For example:
- `c`: Capitalize the first letter  
- `$1$2`: Add "12" at the end  
- `Az`: Append "z"  
- `r`: Reverse the password  
- `c$123`: Capitalize and add "123"  
- `u`: Capitalize the entire word  
- `d`: Duplicate the password  
- `$!`: Add "!" at the end  
- `c$2024`: Capitalize and append the year  
- `$1`: Add "1" at the end  

These rules simulate common user behaviors when creating passwords, improving the chances of successful cracking during dictionary attacks.


## Hydra

![Hash passwords](imag/hydra_1.jpeg)

In this image, we executed the Hydra tool to perform a brute-force attack against the DVWA login page. The command specifies a list of usernames (`users.txt`) and a custom password list (`custom_passwords.txt`), targeting the login form at `192.168.100.217`. The `http-post-form` option defines the structure of the login request and the string that indicates a failed login attempt. Hydra begins testing different combinations, and several successful login attempts for the `admin` user are displayed, including passwords such as `admin`, `adminpa`, `password`, and `123456`.

![Hash passwords](imag/hydra_2.jpeg)

This shows Hydra continuing the brute-force process for additional users such as `gordonb` and `1337pablo`. The output confirms multiple successful password guesses using weak or default credentials like `admin`, `DVWA`, `Hello`, and `user1`. Each green line indicates a valid login found by Hydra, demonstrating how easily weak passwords can be exploited across multiple user accounts.

![Hash passwords](imag/hydra_3.jpeg)

Hydra completes the brute-force session and summarizes the results. The report confirms that the tool successfully tested all credentials and identified **59 valid username-password combinations** across the users listed. This demonstrates the effectiveness of Hydra when combined with a targeted user list and a well-constructed custom wordlist, and emphasizes the importance of strong password policies in preventing such attacks.

## ModSecurity Rules

![Hash passwords](imag/regperso.jpeg)

This image displays the custom rules to detect and block brute force login attempts.

The first rule matches access to `/DVWA/login.php` and checks the response for the string `"Login failed"`. If that condition is met, a counter (`ip.brute_force`) is incremented for the attacking IP address. The counter expires after 60 seconds.

The second rule triggers when the value of `ip.brute_force` exceeds 5, meaning the IP has failed more than five login attempts within a minute. In that case, the request is blocked and logged as a brute force attack.

These rules provide automated detection and mitigation of brute force attempts on the login form.

![Hash passwords](imag/ms_1.jpeg)

This image shows the base configuration for enabling ModSecurity. The rule engine is turned on with `SecRuleEngine On`, and a specific Core Rule Set (CRS) file is included using `Include /etc/modsecurity/crs/REQUEST-949-BLOCKING-EVALUATION.conf`. The line `SecStatusEngine On` is also added to enable logging and status visibility.

This configuration is activating ModSecurity, allowing it to monitor web traffic and apply the defined security rules. Without this setup, the custom rules written later would not be executed.

![Hash passwords](imag/m_s_2.jpeg)

This is the execution of a brute force attack using the Hydra tool on the DVWA login page. The command specifies the use of a username list (`users.txt`) and a custom password list (`custom_passwords.txt`) to attempt multiple login combinations via an HTTP POST request. The target URL is `/DVWA/login.php`, and the attack checks for the response string `"Login failed"` to identify unsuccessful login attempts. As shown in the output, Hydra completed the attack with 60 login attempts, but none of the credentials were valid.

## Comparative analysis of medium vs. hard security levels

![Hash passwords](imag/hvm_1.jpeg)

![Hash passwords](imag/hvm_2.jpeg)

![Hash passwords](imag/hvm_3.jpeg)

### DVWA: Medium vs Hard

| **Feature**                          | **Medium Security Level**                                                                 | **Hard (High) Security Level**                                                                 |
|--------------------------------------|--------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------|
| **Brute Force Login**                | The login form does not change much. No strong protection. Hydra attacks are possible.     | Uses CSRF tokens and stricter session handling. Hydra attacks usually fail.                    |
| **Error Messages**                   | Clear feedback such as `"Login failed"` helps attackers identify failed attempts.          | Generic or no error messages. Sometimes redirects without explanation or blocks IPs.           |
| **CSRF Tokens**                      | Tokens may be present but are often reusable or less restrictive.                          | Tokens change with every session/request, which breaks automated attacks.                      |
| **Login Response Delay or Blocking**| No delay or lockout after multiple failed attempts.                                        | Possible response delay or temporary blocking after several failed attempts.                   |
| **Hidden Defenses (JS, Hidden Fields)** | Minimal hidden protection.                                                                | Extra hidden fields, JavaScript validations, or dynamic tokens complicate automation.          |
| **Hydra Feedback**                   | Hydra clearly detects `"Login failed"` and can brute-force credentials easily.             | Hydra fails due to dynamic tokens and vague error handling.                                    |
| **Evidence of Protection Activation**| No WAF rule triggers or blocking mechanisms observed.                                      | Logs in `/var/log/apache2/modsec_audit.log` show triggered rules and blocked IP addresses.     |



![Hash passwords](imag/ms_3.jpeg)

When attempting to bypass DVWA's "hard" level security using Hydra, the attack was unsuccessful. The tool launched 60 login attempts using a combination of usernames and passwords, but all were rejected. This happened because, at the "hard" security level, DVWA implements stronger defenses such as dynamic CSRF tokens (`user_token`) that change with every request. Since Hydra does not handle these dynamic tokens, it submits outdated or missing tokens, causing all login attempts to fail. Additionally, error messages are less explicit (e.g., no consistent "Login failed" text), which makes it harder for Hydra to detect failed attempts. This confirms that DVWA's hard level effectively mitigates automated brute force attacks.

## Effective countermeasures for each attack vector

### 1. Password Hash Cracking (John the Ripper or Hashcat)

**Attack Vector**:  
Extraction and offline cracking of MD5-hashed passwords using tools like John the Ripper and Hashcat.

**Countermeasures**:
- Use strong hash algorithms like **bcrypt**, **Argon2**, or **scrypt** (avoid MD5/SHA1).
- Apply **unique salts** to each password before hashing.
- Enforce **strong password policies** (length, complexity, expiration).
- Rate-limit login attempts to reduce attack surface even if hashes are leaked.

---

### 2. Weak Wordlists and Custom Rules

**Attack Vector**:  
Attackers optimize their wordlists and apply rules to generate common variations of passwords.

**Countermeasures**:
- Use breached-password detection **APIs** to reject common or exposed credentials.
- Implement **multi-factor authentication (MFA)** to protect against password-only access.
- Configure **lockouts or delay mechanisms** after a number of failed login attempts.

---

### 3. Brute Force via Hydra (HTTP POST)

**Attack Vector**:  
Automated brute force attacks via login forms using tools like Hydra.

**Countermeasures**:
- Add a **CAPTCHA** after several failed login attempts.
- Use **WAFs like ModSecurity** or tools like `fail2ban` to monitor and **block IPs**.
- Log and **alert on suspicious login behavior** such as repeated failures.

---

### 4. Weak Authentication in DVWA (Medium vs. Hard)

**Attack Vector**:  
Exploiting weak token/session validation and verbose error messages in medium security mode.

**Countermeasures**:
- Use **CSRF tokens** that are validated on the server and expire frequently.
- Enforce proper **session expiration and regeneration** on login.
- Set the following **cookie flags** to protect session data:

  - `Secure`: Ensures that the cookie is **only transmitted over HTTPS** connections (SSL/TLS encryption), and **never sent in plaintext** over HTTP. This prevents interception by attackers during transit.
  
  - `HttpOnly`: Instructs the browser to **prevent client-side scripts (e.g., JavaScript)** from accessing the cookie. This mitigates the risk of **cross-site scripting (XSS)** attacks from stealing session tokens. *(Only effective if the browser supports it.)*
  
  - `SameSite`: Tells the browser **how to handle the cookie during cross-site requests**. This can prevent cookies from being sent in CSRF attacks, depending on the configuration (`Strict`, `Lax`, or `None`).

These flags provide **additional security for session cookies**, reducing the chances of session hijacking or misuse by malicious actors.

---

### 5. Attempt to Bypass “Hard” Level Protections

**Attack Vector**:  
Advanced techniques such as manipulating sessions, logic flaws, or parameter tampering to bypass defenses.

**Countermeasures**:
- Perform **regular penetration testing** to uncover flaws in logic and authentication flow.
- Use security-aware frameworks that handle tokens, sessions, and CSRF validation.
- Always keep software and dependencies **up to date**.

---

### 6. Authentication Enumeration

**Attack Vector**:  
Guessing valid usernames based on response message differences or timing.

**Countermeasures**:
- Return **generic error messages** for all failed login attempts.
- Ensure **consistent response time** regardless of username validity.
- Monitor and alert for enumeration patterns via logs.

---

### Conclusion

DVWA's hard-level security configuration demonstrates how a combination of **token rotation**, **session control**, **error message obfuscation**, and **WAF rules** can dramatically increase resistance to brute force and password cracking attacks. When combined with tools like **ModSecurity** and proper password management policies, systems can be made significantly more resilient to real-world authentication threats.
